In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
import time

In [2]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']

In [152]:
for ticker in tickers:
    if ticker != 'NVDA':
        continue
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        #balance = stock.balance_sheet.loc[['Net PPE', 'Total Debt']]
        balance = stock.balance_sheet
        balance = balance.iloc[:, :-1]

        financials = stock.financials
        #earnings = stock.income_stmt.loc[['Operating Revenue', 'Diluted EPS', 'Net Income']]
        earnings = stock.income_stmt

        #cashflow = stock.cash_flow.loc[['Free Cash Flow']]
        cashflow = stock.cash_flow
        cashflow = cashflow.iloc[:, :-1]

        start_dates = balance.columns.values
        end_dates = start_dates + np.timedelta64(366, 'D')

        start_dates = [str(date)[: -19] for date in start_dates]
        end_dates = [str(date)[: -19] for date in end_dates]
        
        year1 = stock.history(start=start_dates[0], end=end_dates[0]).reset_index(drop=True)
        year2 = stock.history(start=start_dates[1], end=end_dates[1]).reset_index(drop=True)
        year3 = stock.history(start=start_dates[2], end=end_dates[2]).reset_index(drop=True)
        year4 = stock.history(start=start_dates[3], end=end_dates[3]).reset_index(drop=True)

        columns = balance.columns.values

        balance.columns = columns
        financials.columns = columns
        earnings.columns= columns
        cashflow.columns = columns

        yearly = pd.concat([balance, earnings, cashflow]).T
        columns = yearly.columns.values

        # Liquidity
        # current ratio = current assets / current liabilities
        yearly['current_ratio'] = yearly['Current Assets'] / yearly['Current Liabilities']
        # quick ratio = Inventory + current assets / current liabilities
        yearly['quick_ratio'] = yearly['Current Assets']
        if 'Inventory' in columns:
            yearly['quick_ratio'] += yearly['Inventory']
        yearly['quick_ratio'] /= yearly['Current Liabilities']
        # cash ratio = (cash + cash equivalents) / current liabilities
        yearly['cash_ratio'] = yearly['Cash Cash Equivalents And Short Term Investments'] / yearly['Current Liabilities']
        yearly['operating_cashflow'] = yearly['Operating Cash Flow']
        # equity ratio = total equity / total assets
        yearly['equity_ratio'] = yearly['Stockholders Equity'] / yearly['Total Assets']

        # Leverage
        # debt to equity = total liabilites / shareholder equity
        yearly['debt_to_equity'] = yearly['Current Liabilities'] / yearly['Stockholders Equity']
        # debt to assets = total liabilites / shareholder equity
        yearly['debt_to_assets'] = yearly['Current Liabilities'] / yearly['Total Assets']
        # interest coverage = operating income / interest expense
        yearly['interest_coverage'] = yearly['Operating Income'] / yearly['Interest Expense']

        # Efficiency
        # asset turnover = revenue / total assets
        yearly['asset_turnover'] = yearly['Total Revenue'] / yearly['Total Assets']
        # receivables turnover = revenue / accounts receivables
        if 'Receivables' in columns: yearly['receivables_turnover'] = yearly['Total Revenue'] / yearly['Receivables']
        else: yearly['receivables_turnover'] = np.nan

        # Profitability
        # gross margin = gross profit / revenue
        yearly['gross_margin'] = yearly['Gross Profit'] / yearly['Total Revenue']
        # profit margin = net income / revenue
        yearly['profit_margin'] = yearly['Net Income'] / yearly['Total Revenue']
        # operating margin = net income / revenue
        yearly['operating_margin'] = yearly['Operating Income'] / yearly['Total Revenue']
        # return on equity = net income / shareholder equity
        yearly['return_on_equity'] = yearly['Net Income'] / yearly['Stockholders Equity']
        # return on assets = net income / total assets
        yearly['return_on_assets'] = yearly['Net Income'] / yearly['Total Assets']
        # eps
        yearly['eps'] = yearly['Diluted EPS']
        shareprice = np.array([year1.loc[0, 'Close'], year2.loc[0, 'Close'], year3.loc[0, 'Close'], year4.loc[0, 'Close']])
        # p/e = shareprice / eps
        yearly['p/e'] = shareprice / yearly['eps']

        yearly['ticker'] = ticker
        yearly['sector'] = info['sector']
        yearly.index = yearly.index.year
        yearly['year'] = yearly.index.values
        
        startprice = np.array([year1.loc[1, 'Open'], year2.loc[1, 'Open'], year3.loc[1, 'Open'], year4.loc[1, 'Open']])
        endprice = np.array([year1.loc[len(year1)-1, 'Close'], year2.loc[len(year2)-1, 'Close'], year3.loc[len(year3)-1, 'Close'], year4.loc[len(year4)-1, 'Close'],])
        pctdiff = (endprice-startprice) / startprice
        yearly['result'] = pctdiff
                
        columns = ["current_ratio", "quick_ratio", "cash_ratio", "operating_cashflow", "equity_ratio", "debt_to_equity", "debt_to_assets", "interest_coverage", "asset_turnover", "receivables_turnover", "gross_margin", "profit_margin", "operating_margin", "return_on_equity", "return_on_assets", "eps", "p/e", "ticker", "sector", "year", "result"]

        yearly = yearly[columns].reset_index(drop=True)
        display(yearly)

        

    except Exception as e:
        print(ticker, e)


,current_ratio,quick_ratio,cash_ratio,operating_cashflow,equity_ratio,debt_to_equity,debt_to_assets,interest_coverage,asset_turnover,receivables_turnover,...,profit_margin,operating_margin,return_on_equity,return_on_assets,eps,p/e,ticker,sector,year,result
0,4.171292,4.66814,2.444173,28090000000.0,0.653877,0.247359,0.161742,128.29572,0.92688,6.092809,...,0.488493,0.541217,0.692447,0.452775,1.193,51.566688,NVDA,Technology,2024,1.081428
1,3.515618,4.301691,2.025903,5641000000.0,0.536667,0.296955,0.159366,21.28626,0.654995,7.048341,...,0.161934,0.206755,0.197638,0.106066,0.174,112.217344,NVDA,Technology,2023,2.126012
2,6.650288,7.251211,4.892272,9108000000.0,0.602259,0.162896,0.098106,42.54661,0.609093,5.787957,...,0.362339,0.373077,0.366451,0.220698,0.385,63.504212,NVDA,Technology,2022,-0.221030
3,4.090446,4.555669,2.945478,5822000000.0,0.586746,0.232345,0.136327,24.630435,0.579174,6.864965,...,0.25979,0.271784,0.256438,0.150464,0.1725,76.553798,NVDA,Technology,2021,0.829080
